# PyTorch Fundamentals

This notebook covers the fundamentals of working with PyTorch for deep learning, including:
- Loading data from Hugging Face
- Applying proper transformations and normalization
- Using pre-trained models (ResNet50)
- Setting up training with appropriate optimization parameters

## Setting Up the Environment

In [ ]:
# Check for or install all necessary packages with conda from environment.yml
%conda env update -f environment.yml

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset
from torchvision import transforms, models
import matplotlib.pyplot as plt
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## Data Loading from Hugging Face

In [ ]:
# Load a dataset from Hugging Face
dataset = load_dataset("cifar10")

# View dataset structure
print(dataset)

## Creating Data Transformations

Data normalization is **critical** for model performance. We use ImageNet statistics for normalization since we'll be using a pre-trained model.

In [ ]:
# Define transformations for training data
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Resize and crop
    transforms.RandomHorizontalFlip(),  # Horizontal flip with 50% probability
    transforms.RandomRotation(15),      # Random rotation up to 15 degrees
    transforms.ToTensor(),              # Convert to tensor
    transforms.Normalize(               # Normalize with ImageNet stats
        mean=[0.485, 0.456, 0.406],     # RGB means from ImageNet
        std=[0.229, 0.224, 0.225]       # RGB standard deviations from ImageNet
    )
])

# Validation transformations (no augmentation, only resize and normalize)
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

## Applying Transformations to Dataset

In [ ]:
# Function to apply transformations to the training dataset
def transform_train_dataset(examples):
    examples["pixel_values"] = [train_transforms(image.convert("RGB")) for image in examples["img"]]
    return examples

# Function to apply transformations to the validation dataset
def transform_val_dataset(examples):
    examples["pixel_values"] = [val_transforms(image.convert("RGB")) for image in examples["img"]]
    return examples

# Apply transformations to training set
transformed_train_dataset = dataset["train"].map(
    transform_train_dataset,
    batched=True,
    remove_columns=["img"]  # Remove original images after transformation
)

# Apply transformations to test set
transformed_val_dataset = dataset["test"].map(
    transform_val_dataset,
    batched=True,
    remove_columns=["img"]
)

# Set the format for PyTorch
transformed_train_dataset.set_format(type="torch", columns=["pixel_values", "label"])
transformed_val_dataset.set_format(type="torch", columns=["pixel_values", "label"])

## Creating DataLoaders

In [ ]:
# Create DataLoaders
train_dataloader = DataLoader(
    transformed_train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4
)

val_dataloader = DataLoader(
    transformed_val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4
)

## Visualizing a Batch

Let's visualize some images from our dataloader to verify transformations are applied correctly.

In [ ]:
# Function to denormalize images for visualization
def denormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return tensor * std + mean

# Get a batch from the dataloader
images, labels = next(iter(train_dataloader))

# Visualize a few images from the batch
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < len(images):
        # Denormalize the image
        img = denormalize(images[i])
        img = img.permute(1, 2, 0).numpy()  # Change from CxHxW to HxWxC
        img = np.clip(img, 0, 1)  # Clip values to valid range
        
        ax.imshow(img)
        ax.set_title(f"Label: {labels[i].item()}")
        ax.axis("off")

plt.tight_layout()
plt.show()

## Model Architecture with ResNet50

In [ ]:
# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer for our task
# For CIFAR-10, we have 10 classes
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Print model architecture summary
print(f"Model: ResNet50")
print(f"Number of trainable parameters: {sum(p.numel() for p in model.fc.parameters() if p.requires_grad)}")

## Training Configuration

We'll use default settings for the optimizer and only tune the learning rate and number of epochs.

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer with default settings
# Only optimize the parameters of the new head (model.fc)
learning_rate = 0.001  # This is one of the few hyperparameters we'll tune
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

# Number of epochs is another parameter we'll tune
num_epochs = 10

## Training Loop

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

# Lists to store metrics for plotting
train_losses = []
train_accs = []
val_losses = []
val_accs = []

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_loss = running_loss / len(train_dataloader)
    train_acc = 100. * correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Validation phase
    model.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    val_loss = val_running_loss / len(val_dataloader)
    val_acc = 100. * correct / total
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    print(f'Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

## Visualizing Training Progress

In [ ]:
# Plot training and validation metrics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
ax1.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
ax1.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True)

# Plot accuracy
ax2.plot(range(1, num_epochs+1), train_accs, label='Train Accuracy')
ax2.plot(range(1, num_epochs+1), val_accs, label='Validation Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Evaluating Model Performance on Test Set

In [ ]:
# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for images, labels in val_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Calculate accuracy for each class
        c = (predicted == labels).squeeze()
        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print(f'Overall Accuracy on the test set: {100 * correct / total:.2f}%')

# Print accuracy for each class
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
for i in range(10):
    print(f'Accuracy of {classes[i]}: {100 * class_correct[i] / class_total[i]:.2f}%')

## Key Points to Remember

1. **Data Normalization is Critical**
   - Always normalize your input data using mean and standard deviation
   - For transfer learning with pre-trained models, use the same normalization values that were used during pre-training (e.g., ImageNet stats)

2. **Data Transformations**
   - Apply appropriate augmentations for training data (flips, rotations, crops)
   - Use only resizing and normalization for validation/test data
   - Transformations help prevent overfitting and improve model generalization

3. **Model Architecture**
   - Start with a pre-trained model like ResNet50
   - Modify only the final layer (head) to match your specific task
   - Freeze pre-trained layers initially to leverage transfer learning

4. **Optimization Settings**
   - Start with default optimizer settings
   - Focus on tuning learning rate and number of epochs first
   - Monitor validation metrics to prevent overfitting

5. **Progressive Unfreezing**
   - After initial training, you can unfreeze more layers gradually
   - Use a smaller learning rate when fine-tuning pre-trained layers

## Next Steps

1. **Hyperparameter Tuning**
   - Try different learning rates
   - Experiment with different batch sizes
   - Test different optimizers (SGD with momentum, AdamW)

2. **Model Improvements**
   - Unfreeze more layers for fine-tuning
   - Try different pre-trained architectures (EfficientNet, ViT)
   - Implement learning rate scheduling

3. **Advanced Techniques**
   - Implement data augmentation strategies like mixup or cutmix
   - Try different loss functions
   - Implement ensemble methods